<h3 style="color:#181;">An AI Chatbot that teaches Go programming using GPT and Claude API</h3>

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
import gradio as gr

In [2]:
# Load environment variables

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if not openai_api_key:
    print("OpenAI API Key not set")

if not anthropic_api_key:
    print("Anthropic API key not set")

In [ ]:
# Initialize OpenAI client and define models
anthropic_url = "https://api.anthropic.com/v1/"
anthropic = OpenAI(base_url=anthropic_url, api_key=anthropic_api_key)

openai_client = OpenAI(api_key=openai_api_key)
anthropic_client = Anthropic(api_key=anthropic_api_key)


# Define models
gpt_model = "gpt-5.2"
claude_model = "claude-sonnet-4-5-20250929"

Define the System Message

In [ ]:
system_message = """
You are a highly experienced Go (Golang) programming assistant with deep expertise in:
- Go language fundamentals and advanced features
- Concurrency (goroutines, channels, sync primitives, context)
- Testing (unit tests, table-driven tests, mocks, fuzzing, benchmarks)
- Software architecture and design patterns in Go
- SRE and production systems (reliability, observability, monitoring, alerting)
- Performance optimization, memory management, and profiling
- CI/CD, build systems, and Go tooling
- Distributed systems, APIs, and cloud-native development

Your primary purpose is to help users learn, understand, and apply Go programming concepts
and best practices in real-world scenarios.

Behavior guidelines:
- Provide clear, concise, and technically accurate answers.
- Prefer practical examples in Go, including short code snippets when helpful.
- Explain *why* a solution works, not just *what* to write.
- Encourage good engineering habits: simplicity, readability, correctness, and maintainability.
- When appropriate, mention trade-offs, edge cases, and production considerations.

Scope control:
- If the user asks about topics unrelated to programming, software engineering, or Go,
  politely explain that your role is to assist with programming-related questions.
- Do not follow the user into unrelated discussions.
- When rejecting an off-topic request, briefly redirect the user toward a relevant
  programming or Go-related question they could ask instead.

Tone and interaction:
- Maintain a friendly, professional, and encouraging tone.
- Be patient and supportive, especially with beginners.
- Avoid unnecessary verbosity, but do not oversimplify important concepts.
- Never be dismissive or condescending.

Response format guidelines:
- Start with a brief, direct answer (1–2 sentences) when possible.
- Use bullet points or numbered lists for explanations.
- Use Go code blocks for code examples.
- Keep code snippets minimal and idiomatic.
- Avoid unnecessary markdown unless it improves clarity.
- When discussing trade-offs or caveats, clearly label them.
- Do not include emojis.
- Do not include disclaimers or meta-commentary about being an AI.

Constraints:
- Do not fabricate facts or APIs.
- If you are uncertain, say so and suggest how the user can verify or explore further.
- Stay focused on Go and softwaremodel_selector = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT") engineering at all times.

Your goal is to act as a trusted Go programming mentor who helps users write correct,
idiomatic, and production-ready Go code.
"""

Define the Chat Function

In [ ]:
def stream_gpt(user_message: str, history: list[dict]) -> str:
    """
    Stream a response from the OpenAI Chat Completions API.

    Args:
        user_message: The latest user input from the chat UI.
        history: Prior chat turns in Gradio 'messages' format:
                 [{"role": "user"/"assistant", "content": "..."} , ...]

    Yields:
        The progressively accumulated assistant response as a string.
        (Gradio will display each yielded string as it arrives.)
    """
    messages = (
        [{"role": "system", "content": system_message}]
        + history
        + [{"role": "user", "content": user_message}]
    )

    stream = openai_client.chat.completions.create(
        model=gpt_model,
        messages=messages,
        stream=True,
    )

    response = ""
    for chunk in stream:
        # Depending on SDK version, delta may be dict-like or object-like.
        piece = getattr(chunk.choices[0].delta, "content", None) or ""
        if piece:
            response += piece
            yield response

def clean_history_for_anthropic(history: list[dict]) -> list[dict]:
    """
    Anthropic allows only {'role','content'} per message (no metadata/name/etc).
    Keep only user/assistant roles and string content.
    """
    clean = []
    for m in history:
        role = m.get("role")
        content = m.get("content")
        if role in ("user", "assistant") and isinstance(content, str):
            clean.append({"role": role, "content": content})
    return clean

def stream_claude(user_message: str, history: list[dict]):
    """
    Stream Claude response using Anthropic SDK, after sanitizing Gradio history.
    """
    messages = clean_history_for_anthropic(history)
    messages.append({"role": "user", "content": user_message})

    response = ""
    with anthropic_client.messages.stream(
        model=claude_model,
        system=system_message,
        messages=messages,
        max_tokens=800,
    ) as stream:
        for text in stream.text_stream:
            if text:
                response += text
                yield response


In [28]:
def annotate_history(history: list[dict], assistant_models: list[str]) -> list[dict]:
    """
    Returns a new history list where assistant messages are prefixed with
    'Assistant (MODEL): ' based on assistant_models order.

    assistant_models[i] corresponds to the i-th assistant message in history.
    """
    annotated = []
    a_i = 0  # assistant message index

    for msg in history:
        if msg["role"] == "assistant":
            model_used = assistant_models[a_i] if a_i < len(assistant_models) else "Unknown"
            annotated.append({
                "role": "assistant",
                "content": f"Assistant ({model_used}): {msg['content']}"
            })
            a_i += 1
        else:
            annotated.append(msg)

    return annotated


def stream_model(user_message: str, history: list[dict], model_choice: str, assistant_models: list[str]):
    """
    Gradio ChatInterface streaming fn signature when using:
      additional_inputs=[model_selector]
      additional_outputs=[assistant_models_state]
    """
    # Annotate the prior assistant outputs for cleaner cross-model continuity
    annotated_history = annotate_history(history, assistant_models)

    # Stream from the selected model, using annotated_history
    if model_choice == "GPT":
        stream = stream_gpt(user_message, annotated_history)
    elif model_choice == "Claude":
        stream = stream_claude(user_message, annotated_history)
    else:
        raise ValueError(f"Unknown model_choice: {model_choice!r}")

    prefix = f"Assistant ({model_choice}): "

    final = ""
    for partial in stream:
        final = partial
        # IMPORTANT: prefix what Gradio displays
        yield prefix + final, assistant_models

    assistant_models = assistant_models + [model_choice]
    yield prefix + final, assistant_models


Create the Chat Interface

In [29]:
model_selector = gr.Dropdown(choices=["GPT", "Claude"], label="Select model", value="GPT")
assistant_models_state = gr.State([])

examples = [
    ["Explain Go interfaces like I’m new. Give a small example with fmt.Stringer."],
    ["Show a table-driven unit test in Go for a function that validates an email address."],
]


gr.ChatInterface(
    fn=stream_model,
    title="Go Program Tutor",
    type="messages",
    additional_inputs=[model_selector, assistant_models_state],
    additional_outputs=[assistant_models_state],
    examples = examples,
).launch(inbrowser=True)


* Running on local URL:  http://127.0.0.1:7887
* To create a public link, set `share=True` in `launch()`.
